In [16]:
import numpy as np
import torch
from preprocess.dataset import get_MNIST,get_dataset,get_handler
from models.model import Model
from al_methods.random_sampling import RandomSampling
from ssl_methods.semi_fixmatch import fixmatch
import models
from torchvision import transforms
from framework.framework2 import Framework2
from copy import deepcopy
import torch.nn as nn
import time

In [17]:
# in the main 
X_tr, Y_tr, X_te, Y_te = get_dataset("Mnist", "./datasets")

In [18]:
# in the main 
if type(X_tr) is list:
    X_tr = np.array(X_tr)
    Y_tr = torch.tensor(np.array(Y_tr))
    X_te = np.array(X_te)
    Y_te = torch.tensor(np.array(Y_te))

if type(X_tr[0]) is not np.ndarray:
    X_tr = X_tr.numpy()
    X_te = X_te.numpy()
    

In [19]:
# in the main
n_pool = len(Y_tr)
n_test = len(Y_te)
#in the main
handler = get_handler("mnist")
# main or framewrok to see
nEnd=100 # total number to query 
nQuery=10 # nombre of points to query in batch 
nStart=10 # nbre of points to start
NUM_INIT_LB = int(nStart*n_pool/100)
NUM_QUERY = int(nQuery*n_pool/100) if nStart!= 100 else 0
NUM_ROUND = int((int(nEnd*n_pool/100) - NUM_INIT_LB)/ NUM_QUERY) if nStart!= 100 else 0
if NUM_QUERY != 0:
    if (int(nEnd*n_pool/100) - NUM_INIT_LB)% NUM_QUERY != 0:
        NUM_ROUND += 1
print(NUM_INIT_LB,NUM_ROUND,NUM_QUERY)
model_name = sorted(name for name in models.__dict__ 
                    if callable(models.__dict__[name]))
model_name
model=Model('resnet50').get_model()
model
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # Conversion en 3 canaux
model.fc = torch.nn.Linear(2048, 10)  # Modifier la couche de classification pour 10 classes
# in the main file
idxs_lb = np.zeros(n_pool, dtype=bool)
idxs_lb
# in the main file
idxs_tmp = np.arange(n_pool)
idxs_tmp
np.random.shuffle(idxs_tmp)
idxs_tmp
# in the main file
idxs_lb[idxs_tmp[:NUM_INIT_LB]] = True
idxs_lb

6000 9 6000


array([False, False,  True, ..., False, False, False])

In [20]:
args_pool = {'mnist':
                { 
                 'n_class':10,
                 'channels':1,
                 'size': 28,
                 'transform_tr': transforms.Compose([
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'transform_te': transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'loader_tr_args':{'batch_size': 128, 'num_workers': 8},
                 'loader_te_args':{'batch_size': 1024, 'num_workers': 8},
                 'normalize':{'mean': (0.1307,), 'std': (0.3081,)},
                },
            
        }

In [21]:
class Args:
    def __init__(self,n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize):
        self.n_class=n_class
        self.img_size=img_size
        self.channels=channels
        self.transform_tr=transform_tr
        self.transform_te=transform_te
        self.loader_tr_args=loader_tr_args
        self.loader_te_args=loader_te_args
        self.normalize=normalize
        self.dataset='mnist'
        self.save_path='./save'
        self.model='ResNet50'
        self.lr=0.1
        self.schedule = [20, 40]
        self.momentum=0.9
        self.gammas=[0.1,0.1]
        self.strategy='framwork1'
        self.optimizer='SGD'
        self.save_model=False

In [22]:
dataset_args = args_pool["mnist"]
n_class = dataset_args['n_class']
img_size = dataset_args['size']
channels = dataset_args['channels']
transform_tr = dataset_args['transform_tr']
transform_te = dataset_args['transform_te']
loader_tr_args = dataset_args['loader_tr_args']
loader_te_args = dataset_args['loader_te_args']
normalize = dataset_args['normalize']

In [23]:
args=Args(n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize)

In [24]:
al_strat=RandomSampling(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args)
ssl_strat=fixmatch(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args)

In [25]:
framework_2= Framework2(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args,al_strat,ssl_strat)

In [26]:
framework_2.train(2e-3,10)

Let's use 1 GPUs!
[Batch=000] [Loss=2.55]
[Batch=010] [Loss=8.00]
[Batch=020] [Loss=4.16]
[Batch=030] [Loss=3.22]
[Batch=040] [Loss=2.58]

==>>[2023-08-11 12:47:00] [Epoch=000/010] [framwork1 Need: 00:00:00] [LR=0.1000] [Best : Test Accuracy=0.00, Error=1.00]
[Batch=000] [Loss=12.50]
[Batch=010] [Loss=3.88]
[Batch=020] [Loss=5.88]
[Batch=030] [Loss=2.08]
[Batch=040] [Loss=2.76]

==>>[2023-08-11 12:47:03] [Epoch=001/010] [framwork1 Need: 00:00:29] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=2.21]
[Batch=010] [Loss=2.22]
[Batch=020] [Loss=2.01]
[Batch=030] [Loss=1.67]
[Batch=040] [Loss=7.36]

==>>[2023-08-11 12:47:06] [Epoch=002/010] [framwork1 Need: 00:00:26] [LR=0.1000] [Best : Test Accuracy=0.15, Error=0.85]
[Batch=000] [Loss=1.68]
[Batch=010] [Loss=1.40]
[Batch=020] [Loss=1.26]
[Batch=030] [Loss=1.29]
[Batch=040] [Loss=1.33]

==>>[2023-08-11 12:47:10] [Epoch=003/010] [framwork1 Need: 00:00:23] [LR=0.1000] [Best : Test Accuracy=0.43, Error=0.57]
[Batch=000] [

0.8425

In [29]:
test_acc=framework_2.strat_1.predict(X_te,Y_te)

In [30]:
test_acc

0.0982

In [ ]:
acc = np.zeros(NUM_ROUND+1)
acc[0] = test_acc

In [ ]:
len(acc)

In [ ]:
# for rd in range(1, NUM_ROUND+1):

#     if rd%2==0:
#         # Al_methods
#         print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
#         labeled = len(np.arange(n_pool)[idxs_lb])
#         if NUM_QUERY > int(nEnd*n_pool/100) - labeled:
#             NUM_QUERY = int(nEnd*n_pool/100) - labeled
            
#         # query
#         ts = time.time()
#         output = al_strat.query(NUM_QUERY)
#         q_idxs = output
#         idxs_lb[q_idxs] = True
#         te = time.time()
#         tp = te - ts
        
#         # update
#         al_strat.update(idxs_lb)
#         best_test_acc = al_strat.train(alpha=2e-3, n_epoch=10)

#         t_iter = time.time() - ts
        
#         # round accuracy
#         # test_acc = strategy.predict(X_te, Y_te)
#         acc[rd] = best_test_acc
#     else:
#         #SSL methods
        
#         print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
#         labeled = len(np.arange(n_pool)[idxs_lb])
#         if NUM_QUERY > int(nEnd*n_pool/100) - labeled:
#             NUM_QUERY = int(nEnd*n_pool/100) - labeled
            
#         # query
#         ts = time.time()
#         output = ssl_strat.query(NUM_QUERY)
#         q_idxs = output
#         idxs_lb[q_idxs] = True
#         te = time.time()
#         tp = te - ts
        
#         # update
#         ssl_strat.update(idxs_lb)
#         best_test_acc = ssl_strat.train(alpha=2e-3, n_epoch=10)

#         t_iter = time.time() - ts
        
#         # round accuracy
#         # test_acc = strategy.predict(X_te, Y_te)
#         acc[rd] = best_test_acc
